# Dataset

In [1]:
run ../utils/preprocess.py

In [2]:
category = "Video_Games"
C.INPUT_DATA_PATH = '../../data/90_input'
with open('%s/train-%s.pickle' % (C.INPUT_DATA_PATH, category), 'rb') as f:
    train_data = pd.read_pickle(f)

In [3]:
train_data.head()

,asin,questionsList,reviewsList
89,B0018YXM3Y,[{'text': 'does this this game run on windows ...,"[{'unhelpful': 7, 'text': 'This is a great gam..."
114,B005761E70,[{'text': 'are the thumstick controls more pre...,"[{'unhelpful': 0, 'text': 'I got this controll..."
419,B00GHTP7SS,[{'text': 'This is only the code? Where do i d...,"[{'unhelpful': 0, 'text': 'the game runs well ..."
172,B001BZ2F56,[{'text': 'Is this software work well with Win...,"[{'unhelpful': 2, 'text': 'I really like tycoo..."
208,B0091WIOJ2,[{'text': 'Do i need a 7.1 Surround sound card...,"[{'unhelpful': 0, 'text': 'my review, my opini..."


In [6]:
vocab = Vocabulary(10000)

In [50]:
def tokenize(text):
    punctuations = string.punctuation.replace("\'", '')
    
    for ch in punctuations:
        text = text.replace(ch, " "+ch+" ")
    
    tokens = text.split()
    
    for i in range(len(tokens)):
        token = tokens[i]
        if token.isupper() == False:
            tokens[i] = token.lower()
    
    return tokens

In [52]:
tokenize("Hi, Hello! CMU. don't ")

['hi', ',', 'hello', '!', 'CMU', '.', "don't"]

In [66]:
for index, row in train_data.iterrows():
    questionsList = row['questionsList']
    for question in questionsList:
        if 'text' in question:
            text = question['text']
            vocab.add_sequence(tokenize(text))

            for answer in question['answers']:
                text = answer['text']
                vocab.add_sequence(tokenize(text))
    
    reviewsList = row['reviewsList']
    for review in reviewsList:
        text = review['text']
        vocab.add_sequence(tokenize(text))

In [68]:
vocab.sort_vocabulary()
vocab._token2index

{'.': 4,
 'the': 5,
 ',': 6,
 'and': 7,
 'to': 8,
 'a': 9,
 'I': 10,
 'it': 11,
 'is': 12,
 'of': 13,
 'this': 14,
 'you': 15,
 'game': 16,
 'for': 17,
 'that': 18,
 'in': 19,
 'with': 20,
 'but': 21,
 'on': 22,
 'have': 23,
 '-': 24,
 'are': 25,
 'my': 26,
 'not': 27,
 'as': 28,
 '!': 29,
 'was': 30,
 'so': 31,
 'if': 32,
 'be': 33,
 'games': 34,
 'like': 35,
 'can': 36,
 'one': 37,
 'all': 38,
 'just': 39,
 ')': 40,
 'play': 41,
 'or': 42,
 '(': 43,
 'your': 44,
 'get': 45,
 'they': 46,
 "it's": 47,
 'great': 48,
 'will': 49,
 'more': 50,
 'at': 51,
 'there': 52,
 'no': 53,
 'from': 54,
 'out': 55,
 'has': 56,
 'up': 57,
 'me': 58,
 'very': 59,
 'good': 60,
 'an': 61,
 'really': 62,
 '?': 63,
 'when': 64,
 'time': 65,
 'only': 66,
 'do': 67,
 'fun': 68,
 ':': 69,
 'some': 70,
 'would': 71,
 '"': 72,
 'much': 73,
 'about': 74,
 'had': 75,
 'xbox': 76,
 "don't": 77,
 'i': 78,
 'what': 79,
 'than': 80,
 '/': 81,
 'new': 82,
 'well': 83,
 'other': 84,
 ';': 85,
 'also': 86,
 'use': 87,
 

In [ ]:
for row in train_data.head(2):
    print(row['questionsList'])
    for question in questionsList:
        for answer in question['answers']:
                print(answer['text'])

In [4]:
class Vocabulary(object):
    """
    A Vocabulary stores a set of words in the corpus mapped to unique integer IDs.

    In addition to the words in the actual language, a Vocabulary includes three
    reserved tokens (and IDs) for the start-of-sequence and end-of-sequence
    markers, and for a special 'UNK' marker used to handle rare/unknown words.

    The Vocabulary is sorted in descending order based on frequency. If the
    number of words seen is greater than the maximum size of the Vocabulary,
    the remaining least-frequent words are ignored.

    Args: size(int): maximum number of words allowed in this vocabulary
    """
    def __init__(self, max_vocab_size):
        self.PAD_token_name = "<PAD>"
        self.UNK_token_name = "<UNK>"
        self.SOS_token_name = "<SOS>"
        self.EOS_token_name = "<EOS>"
        self.PAD_token_id = 0
        self.UNK_token_id = 1
        self.SOS_token_id = 2
        self.EOS_token_id = 3

        self._reserved = set([self.PAD_token_name, self.UNK_token_name, \
                self.SOS_token_name, self.EOS_token_name])
        self._reserved_token_id = [
                (self.PAD_token_name, self.PAD_token_id),
                (self.UNK_token_name, self.UNK_token_id),
                (self.SOS_token_name, self.SOS_token_id),
                (self.EOS_token_name, self.EOS_token_id)
        ]

        self._token2index = dict([(tok, idx) for tok, idx in self._reserved_token_id])
        self._index2token = dict([(idx, tok) for tok, idx in self._reserved_token_id])

        self._token2count = {}

        self._num_tokens = 0
        self._num_reserved = 4
        self.sorted = False
        self.size = max_vocab_size

    def trim(self):
        """
        Sorts the vocabulary in descending order based on frequency
        """
        sorted_vocab_count = sorted(self._token2count.items(), \
                key=lambda x: x[1], reverse=True)[:self.size]
        self._token2index = dict( [ (w, self._num_reserved + idx) \
                for idx, (w, _) in enumerate(sorted_vocab_count) ] )
        self._index2token = dict( [ (idx, w) \
                for w, idx in self._token2index.items() ])

        for tok, idx in self._reserved_token_id:
            self._token2index[tok] = idx
            self._index2token[idx] = tok

        if self._num_tokens > self.size:
            self._num_tokens = self.size

        self.sorted = True

    def sort_vocabulary(self):
        """
        Sorts the vocabulary (if it is not already sorted).
        """
        if not self.sorted:
            self.trim()


    def get_index(self, token):
        """
        Returns: int: ID of the given token.
        """
        self.sort_vocabulary()
        return self._token2index[token]


    def get_token(self, index):
        """
        Returns: str: token with ID equal to the given index.
        """
        self.sort_vocabulary()
        return self._index2token[index]


    def get_vocab_size(self):
        """
        Returns: int: maximum number of words in the vocabulary.
        """
        self.sort_vocabulary()
        return self._num_tokens + self._num_reserved


    def add_token(self, token):
        """
        Adds an occurrence of a token to the vocabulary,
        incrementing its observed frequency if the word already exists.
        Args: token (int): word to add
        """
        if token in self._reserved:
            return
        if token not in self._token2count:
            self._token2count[token] = 1
            self._num_tokens += 1
        else:
            self._token2count[token] += 1
        self.sorted = False

    def add_sequence(self, sequence):
        """
        Adds a sequence of words to the vocabulary.
        Args: sequence(list(str)): list of words, e.g. representing a sentence.
        """
        for tok in sequence:
            self.add_token(tok)

    def indices_from_sequence(self, sequence):
        """
        Maps a list of words to their token IDs, or else <UNK>
        if the word is rare/unknown.
        Args: sequence (list(str)): list of words to map
        Returns: list(int): list of mapped IDs
        """
        self.sort_vocabulary()
        return [self._token2index[tok]
                if tok in self._token2index
                else self.UNK_token_id
                for tok in sequence]

    def sequence_from_indices(self, indices):
        """
        Recover a sentence from a list of token IDs.
        Args: indices (list(int)): list of token IDs.
        Returns: list(str): recovered sentence, represented as a list of words
        """
        seq = [self._index2token[idx] for idx in indices]
        return seq


    def __eq__(self, other):
        if not isinstance(other, self.__class__):
            return False
        self.sort_vocabulary()
        other.sort_vocabulary()

        if self._token2count == other._token2count \
                and self._token2index == other._token2index \
                and self._index2token == other._index2token:
            return True
        else:
            return False

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        return hash(self._token2index)


# DataLoader

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

import numpy as np
from operator import itemgetter, attrgetter
from torch.utils.data import Dataset, DataLoader

In [ ]:
reviews1 = [[1,2], [1,2,3],[6,7,8]]
reviews2 = [[1], [1,2]]

question1 = [1,2,3]
question2 = [2,3,4,5]

answer1 = [1,3]
answer2 = [2]

data1 = [(answer1), (answer2)]
data2 = [(answer1, question1), (answer2, question2)]
data3 = [(answer1, question1, reviews1), (answer2, question2, reviews2)]

In [ ]:
print(data1)
print(data2)
print(data3)

In [ ]:
a, q = zip(*data2)
print(a)
print(q)

In [ ]:
class AmazonDataLoader(DataLoader):

    def sortByLength(self, item):
        if self.mode is "1":
            return len(item)
        
        elif self.mode is "2":
            assert(len(item) == 2)
            return len(item[0])
        
        elif self.mode is "3":
            assert(len(item) == 3)
            reviews = item[2]
            max_len = 0
            for review in reviews:
                max_len = max(max_len, len(review))
            return max_len

        
    def __init__(self, data, mode, batch_size):
        self.batch_size = batch_size
        self.mode = mode
        
        sorted(data, key=self.sortByLength, reverse=True)    
        self.data = data
        

    def create_packed_qa(self, batch_data):
        lengths = np.array([len(item) for item in batch_data])
        max_len = max(lengths)
        
        padded_data = np.array( [np.pad(item, (0, max_len-len(item)), 'constant') for item in batch_data] )
        padded_data = torch.from_numpy(padded_data)
        
        return (padded_data)
    
    def create_packed_reviews(self, review_data):
        max_num_reviews = 0
        for reviews in review_data:
            max_num_reviews = max(max_num_reviews, len(reviews))
        
        data = []
        for i in range(max_num_reviews):
            batch_data = []
            for j in range(self.batch_size):
                reviews = review_data[j]
                if i < len(reviews):
                    batch_data.append(reviews[i])
                else:
                    batch_data.append([0])
            data.append(batch_data)
        
        padded_data = []
        for i in range(max_num_reviews):
            batch_data = data[i]
            lengths = [len(review) for review in batch_data]
            max_len = max(lengths)
            
            padded_batch_data = np.array([np.pad(item, (0, max_len-len(item)), 'constant') for item in batch_data])
            padded_batch_data = torch.from_numpy(padded_batch_data)
            
            padded_data.append(padded_batch_data)
            
        return padded_data
        
    def __iter__(self):
        print(self.data)
        self.num_batches = len(self.data) // self.batch_size
        indices = np.arange(self.num_batches)
        np.random.shuffle(indices)

        for index in indices:
            start = index*self.batch_size
            end = (index+1)*self.batch_size
            
            batch_data = self.data[start:end]
            assert(self.batch_size == len(batch_data))
            
            if self.mode is "1":
                answers = batch_data
                packed_answers = self.create_packed_qa(answers)
                yield (packed_answers)
                
            elif self.mode is "2":
                answers, questions = zip(*batch_data)
                packed_answers = self.create_packed_qa(list(answers))
                packed_questions = self.create_packed_qa(list(questions))
                yield (packed_answers, packed_questions)
            
            elif self.mode is "3":
                answers, questions, reviews = zip(*batch_data)
                packed_answers = self.create_packed_qa(list(answers))
                packed_questions = self.create_packed_qa(list(questions))
                packed_reviews = self.create_packed_reviews(list(reviews))
                yield (packed_answers, packed_questions, packed_reviews)
                
    def __len__(self):
        return self.num_batches

In [ ]:
data_loader = AmazonDataLoader(data3, "3", 2)

In [ ]:
for batch_idx, data in enumerate(data_loader):
    answers, questions, reviews = data
    print(data)

In [ ]:
def sortByMaxReviewLength(item):
    reviews = item[2]
    max_len = 0
    for review in reviews:
        max_len = max(max_len, review.shape[0])
    return max_len

for item in data3:
    assert(len(item) == 3)

sorted(data3, key=sortByMaxReviewLength, reverse=True)

In [ ]:
def sortByAnswerLength(item):
    return item[0].shape[0]

sorted(data2, key=sortByAnswerLength, reverse=True)

In [ ]:
answers = data1
print(answers)

answers, questions = zip(*data2)
print(data2)
print(answers)
print(questions)

answers, questions, reviews = zip(*data3)
print(data3)
print(answers)
print(questions)
print(reviews)

In [ ]:
x = Variable(torch.randn(10, 10, 30))
lens = list(range(1, 11))
x = pack_padded_sequence(x, lens[::-1], batch_first=True)
y = pad_packed_sequence(x)